In [3]:
#hashlib

import hashlib
m = hashlib.sha256()
m.update(b"Nobody inspects")
m.update(b" the spammish repetition")
m.digest()

m.hexdigest()

hashlib.sha256(b"Nobody inspects the spammish repetition").hexdigest()

'031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406'

In [4]:
#Constructors
h = hashlib.new('sha256')
h.update(b"Nobody inspects the spammish repetition")
h.hexdigest()

'031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406'

In [5]:
#SHAKE variable digests
h = hashlib.shake_256(b'Nobody inspects the spammish repetition')
h.hexdigest(20)

'44709d6fcb83d92a76dcb0b668c98e1b1d3dafe7'

In [9]:
#File Hashing
import io, hashlib, hmac
with open(hashlib.__file__, "rb") as f:
    digest = hashlib.file_digest(f, "sha256")

digest.hexdigest()  

#>>>#

buf = io.BytesIO(b"somedata")
mac1 = hmac.HMAC(b"key", digestmod=hashlib.sha512)
digest = hashlib.file_digest(buf, lambda: mac1)

#>>>#

digest is mac1

mac2 = hmac.HMAC(b"key", b"somedata", digestmod=hashlib.sha512)
mac1.digest() == mac2.digest()

AttributeError: module 'hashlib' has no attribute 'file_digest'

In [10]:
### ^ gives attribute Error: module 'hashlib' has no attribute 'file_digest'

In [11]:
#Use buffers for memory
import sys
import hashlib

# BUF_SIZE is totally arbitrary, change for your app!
BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

md5 = hashlib.md5()
sha1 = hashlib.sha1()

with open(sys.argv[1], 'rb') as f:
    while True:
        data = f.read(BUF_SIZE)
        if not data:
            break
        md5.update(data)
        sha1.update(data)

print("MD5: {0}".format(md5.hexdigest()))
print("SHA1: {0}".format(sha1.hexdigest()))

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [12]:
# ^ FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [13]:
#Key Derivation
from hashlib import pbkdf2_hmac
our_app_iters = 500_000  # Application specific, read above.
dk = pbkdf2_hmac('sha256', b'password', b'bad salt'*2, our_app_iters)
dk.hex()

#uses iteration, comes standard with SSL now,

'15530bba69924174860db778f2c6f8104d3aaf9d26241840c8c4a641c8d000a9'

In [14]:
#BLAKE2
#RFC7693

from hashlib import blake2b
h = blake2b()
h.update(b'Hello world')
h.hexdigest()

#As a shortcut, you can pass the first chunk of data to update directly to the constructor as the positional argument:

#>>>#

from hashlib import blake2b
blake2b(b'Hello world').hexdigest()

#You can call hash.update() as many times as you need to iteratively update the hash:

#>>>#

from hashlib import blake2b
items = [b'Hello', b' ', b'world']
h = blake2b()
for item in items:
    h.update(item)
h.hexdigest()
'6ff843ba685842aa82031d3f53c48b66326df7639a63d128974c5c14f31a0f33343a8c65551134ed1ae0f2b0dd2bb495dc81039e3eeb0aa1bb0388bbeac29183'

#Using different digest sizes
#BLAKE2 has configurable size of digests up to 64 bytes for BLAKE2b and up to 32 bytes for BLAKE2s. For example, to replace SHA-1 with BLAKE2b without changing the size of output, we can tell BLAKE2b to produce 20-byte digests:

#>>>#

from hashlib import blake2b
h = blake2b(digest_size=20)
h.update(b'Replacing SHA1 with the more secure function')
h.hexdigest()
'd24f26cf8de66472d58d4e1b1774b4c9158b1f4c'
h.digest_size
20
len(h.digest())
20

#Hash objects with different digest sizes have completely different outputs (shorter hashes are not prefixes of longer hashes); BLAKE2b and BLAKE2s produce different outputs even if the output length is the same:

#>>>#

from hashlib import blake2b, blake2s
blake2b(digest_size=10).hexdigest()
'6fa1d8fcfd719046d762'
blake2b(digest_size=11).hexdigest()
'eb6ec15daf9546254f0809'
blake2s(digest_size=10).hexdigest()
'1bf21a98c78a1c376ae9'
blake2s(digest_size=11).hexdigest()
'567004bf96e4a25773ebf4'

'567004bf96e4a25773ebf4'

In [16]:
#Keyed hashing in Blake2

from hashlib import blake2b
h = blake2b(key=b'pseudorandom key', digest_size=16)
h.update(b'message data')
h.hexdigest()

#As a practical example, a web application can symmetrically sign cookies sent to users and later verify them to make sure they weren’t tampered with:

#>>>#

from hashlib import blake2b
from hmac import compare_digest

SECRET_KEY = b'pseudorandomly generated server secret key'
AUTH_SIZE = 16

def sign(cookie):
    h = blake2b(digest_size=AUTH_SIZE, key=SECRET_KEY)
    h.update(cookie)
    return h.hexdigest().encode('utf-8')

def verify(cookie, sig):
    good_sig = sign(cookie)
    return compare_digest(good_sig, sig)

cookie = b'user-alice'
sig = sign(cookie)
print("{0},{1}".format(cookie.decode('utf-8'), sig))

verify(cookie, sig)

verify(b'user-bob', sig)

verify(cookie, b'0102030405060708090a0b0c0d0e0f00')
#Even though there’s a native keyed hashing mode, BLAKE2 can, of course, be used in HMAC construction with hmac module:

#>>>#

import hmac, hashlib
m = hmac.new(b'secret key', digestmod=hashlib.blake2s)
m.update(b'message')
m.hexdigest()

user-alice,b'43b3c982cf697e0c5ab22172d1ca7421'


'e3c8102868d28b5ff85fc35dda07329970d1a01e273c37481326fe0c861c8142'

In [17]:
#Random hashing, salted hashing or just hashing
#really not recommended for use with Blake2 -> not suitable for hashing passwords

import os
from hashlib import blake2b
msg = b'some message'
# Calculate the first hash with a random salt.
salt1 = os.urandom(blake2b.S#ALT_SIZE)
h1 = blake2b(salt=salt1)
h1.update(msg)
# Calculate the second hash with a different random salt.
salt2 = os.urandom(blake2b.SALT_SIZE)
h2 = blake2b(salt=salt2)
h2.update(msg)
# The digests are different.
h1.digest() != h2.digest()

True

In [19]:
#Personalization

#lowers probability of breaking attack
from hashlib import blake2b
FILES_HASH_PERSON = b'MyApp Files Hash'
BLOCK_HASH_PERSON = b'MyApp Block Hash'
h = blake2b(digest_size=32, person=FILES_HASH_PERSON)
h.update(b'the same content')
h.hexdigest()

h = blake2b(digest_size=32, person=BLOCK_HASH_PERSON)
h.update(b'the same content')
h.hexdigest()

#Personalization together with the keyed mode can also be used to derive different keys from a single one.

#>>>#

from hashlib import blake2s
from base64 import b64decode, b64encode
orig_key = b64decode(b'Rm5EPJai72qcK3RGBpW3vPNfZy5OZothY+kHY6h21KM=')
enc_key = blake2s(key=orig_key, person=b'kEncrypt').digest()
mac_key = blake2s(key=orig_key, person=b'kMAC').digest()
print(b64encode(enc_key).decode('utf-8'))

print(b64encode(mac_key).decode('utf-8'))

rbPb15S/Z9t+agffno5wuhB77VbRi6F9Iv2qIxU7WHw=
G9GtHFE1YluXY1zWPlYk1e/nWfu0WSEb0KRcjhDeP/o=


In [20]:
#This example uses 64-byte internal digests, and returns the 32-byte final digest:

from hashlib import blake2b

FANOUT = 2
DEPTH = 2
LEAF_SIZE = 4096
INNER_SIZE = 64

buf = bytearray(6000)

# Left leaf
h00 = blake2b(buf[0:LEAF_SIZE], fanout=FANOUT, depth=DEPTH,
              leaf_size=LEAF_SIZE, inner_size=INNER_SIZE,
              node_offset=0, node_depth=0, last_node=False)
# Right leaf
h01 = blake2b(buf[LEAF_SIZE:], fanout=FANOUT, depth=DEPTH,
              leaf_size=LEAF_SIZE, inner_size=INNER_SIZE,
              node_offset=1, node_depth=0, last_node=True)
# Root node
h10 = blake2b(digest_size=32, fanout=FANOUT, depth=DEPTH,
              leaf_size=LEAF_SIZE, inner_size=INNER_SIZE,
              node_offset=0, node_depth=1, last_node=True)
h10.update(h00.digest())
h10.update(h01.digest())
h10.hexdigest()

'3ad2a9b37c6070e374c7a8c508fe20ca86b6ed54e286e93a0318e95e881db5aa'

In [21]:
#hashlib ^ and hmac
# secrets and token generation, nothing too complex